<a id='Top'></a>

# Intermediate Step - Feature Creation


                                                   By: Jun Ho Lee
                                                   
___


<a id='Table of Contents'></a>
## Table of Contents

1. <a href='#Import Dataset'>Importing the Cleaned Dataset</a>
2. <a href='#Interactivity Introduction'>Introducing Interactivity with Plotly</a>


<a href='#Top'>Back to Top</a>

<a id='Import Dataset'></a>
____
### 1. Importing the Cleaned Dataset

Using the cleaned parameters saved in JSON format, we will load in the cleaned dataset that was saved from the previous notebook. 



**A. Import Necessary Libraries:**


In [1]:
import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as plt
import seaborn as sns
# # set option to see all the columns
pd.set_option('display.max_columns', 50)

# # suppress scientific notation
pd.set_option('display.float_format', lambda x: '%.5f' % x)


**B. Read in JSON file to extract parameters:**

In [2]:
# Import JSON file for column information extraction 

with open('data/H-1B_Cleaned_Params.json', 'r') as fp:
    h1b_params = json.load(fp)

final_cols = h1b_params['final_cols']
column_types = h1b_params['col_types']
date_columns = h1b_params['date_cols']

**C. Read in csv file**

In [3]:
h1b_df = pd.read_csv("data/H-1B_Cleaned_Dataset.csv", usecols = final_cols, dtype=column_types, low_memory=False, parse_dates=date_columns, infer_datetime_format=True)

**D. Verify Memory Usage and column data types**

In [4]:
# # Memory Usage 
# h1b_df.info(memory_usage='deep')

# # Columns
# h1b_df.columns

# # Column dtypes
# h1b_df.dtypes

<a href='#Table of Contents'>Back to Table of Contents</a>

<a id='Feature Creation'></a>
____
### 2. Feature Selection / Creation - Introduction

For machine learning algorithms to perform well and be generalizable, a single feature shouldn't have too may factors. Using ***ipywidgets*** interactivity feature, we will quickly look through each categorical column to see how many *unique* values each column (feature) has and the *value_counts* of that column.



**A. Import ipywidgets interactivity library**

In [5]:
from IPython.display import Image, display, HTML
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
# import plotly
# import plotly.graph_objects as go
# import plotly.express as px  # # px only works with tidy datasets

**B. Using ipywidgets to interactively explore the dataset**

**Purpose:** Outputs ***Description*** of a column of your choosing.

In [6]:
@interact
def describe(column=list(h1b_df.select_dtypes(include=['category','object']).columns)):
    print(h1b_df[column].describe())

interactive(children=(Dropdown(description='column', options=('CASE_STATUS', 'VISA_CLASS', 'EMPLOYER_NAME', 'E…

**Purpose:** Output the ***value_counts*** of a column of your choosing.

In [7]:
@interact
def value_counts(column=list(h1b_df.select_dtypes(include=['category','object']).columns)):
    print(h1b_df[column].value_counts())

interactive(children=(Dropdown(description='column', options=('CASE_STATUS', 'VISA_CLASS', 'EMPLOYER_NAME', 'E…

 **Observations:** Going through all the categorical and object columns, we can see that most of the columns have 100 or less unique values. Some columns however have hundreds of unique values. The presence of hundreds of unique values will later affect our machine learning models to generalize, leading to serious overfitting issues. Therefore we need to categorize these values into distinct non-overlapping categories. Here is a list of columns that have over 100 unique values and what I intend to do with them:

1. `EMPLOYER_NAME` (68992 unique values): 
2. `EMPLOYER_POSTAL_CODE` (10568 unique values): delete column since information is already preserved in `EMPLOYER_STATE_MAPPED` column. 
3. `SOC_CODE`* (736 unique values): Categorize into x different *occupations* based on the nature of the job (inspired by [Datacamp Post](https://www.datacamp.com/community/tutorials/predicting-H-1B-visa-status-python) but referred to [SOC Code Manual](https://www.bls.gov/soc/2018/soc_2018_manual.pdf))
4. `NAICS_CODE` (2026 unique values): categorize according to [NAICS Code Manual](https://www.census.gov/eos/www/naics/2017NAICS/2017_NAICS_Manual.pdf)
5. `WORKSITE_POSTAL_CODE` (14660 unique values): delete column as information preserved in `WORKSITE_STATE` column.  


**`SOC_NAME`* is the job title category that Department of Labor uses to classify the nominal `JOB_TITLE`s.**

<a href='#Table of Contents'>Back to Table of Contents</a>

<a id='Drop Columns'></a>
____
### 2a. Feature Selection / Creation - Part 1

##### Dropping `EMPLOYER_POSTAL_CODE` and `WORKSITE_POSTAL_CODE` 


**A. Drop `POSTAL_CODE` columns**

In [8]:
h1b_dviz_v1 = h1b_df.drop(['EMPLOYER_POSTAL_CODE','WORKSITE_POSTAL_CODE'], axis=1)

# # Verification
h1b_dviz_v1.shape

(616065, 21)

<a href='#Table of Contents'>Back to Table of Contents</a>

<a id='Drop Columns'></a>
____
### 2a. Feature Selection / Creation - Part 2

##### Categorizing `SOC_CODE`

According to the official government manual on [`SOC-CODE`](https://www.bls.gov/soc/2018/soc_2018_manual.pdf), there are 23 major groups which are further divided into 98 minor groups $\rightarrow$ 459 broad occupations $\rightarrow$ and finally 867 detailed occupations. Major groups and minor groups are separated by hyphens ('-'). All major groups are odd-numbered and cannot contain any strings. All codes should be 6 digits total (2 digits - 4 digits). Any code that does not adhere to the template above is an invalid code. To get clean codes, we therefore need to clean the column a little bit. 


**A. Initial Investigation**

In [9]:
# # Copy of original h1b_dviz_v1 into h1b_dviz_v2 to save the cleaned column
h1b_dviz_v2 = h1b_dviz_v1.copy()

# # Change dtype to string because working with categoricals is hard (adding and removing categories is cumbersome)
h1b_dviz_v2['SOC_CODE'] = h1b_dviz_v2.SOC_CODE.astype(str)

# # Investigate unique elements of the column (commented out to save space)
# h1b_dviz_v2.SOC_CODE.unique()

**B. Selecting out first 7 characters to return (xx-xxxx) format**

In [10]:
# # Return first 7 charactrs
h1b_dviz_v2.SOC_CODE = h1b_dviz_v1.SOC_CODE.str[:7]

**C. Dropping those rows where the third element of SOC_CODE is not a hyphen**

- will eliminate any code with strings or with invalid delimiters
- notice the not (~) operator in the code

In [11]:
# # third element is in [2] position
invalid_soc_code_idx = np.where(~(h1b_dviz_v2.SOC_CODE.str[2] == '-'))[0]

# # Drop the invalid indices and reset the index! 
h1b_dviz_v2 = h1b_dviz_v2.drop(invalid_soc_code_idx)
h1b_dviz_v2b = h1b_dviz_v2.reset_index(drop=True)

# # check that the rows were dropped (original length: 654360)
h1b_dviz_v2b.shape

(615465, 21)

**D. Replacing Invalid Delimiters to zero**

[Reference](https://stackoverflow.com/questions/3411771/best-way-to-replace-multiple-characters-in-a-string)

In [12]:
# # Replace invalid delimiters to 0 ('0')
h1b_dviz_v2b.SOC_CODE = h1b_dviz_v2b.SOC_CODE.str.replace('.','0')
# h1b_dviz_v2.SOC_CODE.unique()

**E. Deleting Invalid major group codes**

- even numbered codes
- code must be within 11.0000 and 55.0000 (thus major group should be an odd number between 11 and 55


In [13]:
# # Parse out major codes
h1b_dviz_v2b['SOC_MAJOR_GROUP'] = h1b_dviz_v2b.SOC_CODE.str[:2]
h1b_dviz_v2b['SOC_MAJOR_GROUP'] = pd.to_numeric(h1b_dviz_v2b['SOC_MAJOR_GROUP'])

# # Picking out the invalid major codes
even_soc = np.where(h1b_dviz_v2b['SOC_MAJOR_GROUP'] % 2 == 0)[0]
invalid_soc = np.where((h1b_dviz_v2b['SOC_MAJOR_GROUP'] > 55) | (h1b_dviz_v2b['SOC_MAJOR_GROUP'] < 11))[0]

all_invalid = np.concatenate((even_soc, invalid_soc))


In [14]:
# # Dropping the invalid index and resetting the indes
h1b_dviz_v2b = h1b_dviz_v2b.drop(all_invalid)
h1b_dviz_v2c = h1b_dviz_v2b.reset_index(drop=True)

# # convert SOC_MAJOR_GROUP back to string
h1b_dviz_v2c['SOC_MAJOR_GROUP'] = h1b_dviz_v2c['SOC_MAJOR_GROUP'].astype('str')


In [21]:
# # check that the rows were dropped (original length: 654360)
h1b_dviz_v2c.shape

(615463, 22)

**F. Mapping SOC codes into occupations for better intepretation**

Mapping is as follows: (Major Groups) - Refer to the [SOC-CODE Manual](https://www.bls.gov/soc/2018/soc_2018_manual.pdf)

- 11: Management Occupations
- 13: Business and Financial Operations Occupations
- 15: Computer and Mathematical Occupations
- 17: Architecture and Engineering Occupations
- 19: Life, Physical, and Social Science Occupations
- 21: Community and Social Service Occupations
- 23: Legal Occupations
- 25: Educational Instruction and Library Occupations
- 27: Arts, Design, Entertainment, Sports, and Media Occupations
- 29 Healthcare Practitioners and Technical Occupations
- 31: Healthcare Support Occupations
- 33: Protective Service Occupations
- 35: Food Preparation and Serving Related Occupations
- 37: Building and Grounds Cleaning and Maintenance Occupations
- 39: Personal Care and Service Occupations
- 41: Sales and Related Occupations
- 43: Office and Administrative Support Occupations
- 45: Farming, Fishing, and Forestry Occupations
- 47: Construction and Extraction Occupations
- 49: Installation, Maintenance, and Repair Occupations
- 51: Production Occupations
- 53: Transportation and Material Moving Occupations
- 55: Military Specific Occupations 



In [22]:
soc_map_dict = {"11": "Management Occupations",
"13": "Business and Financial Operations Occupations",
"15": "Computer and Mathematical Occupations",
"17": "Architecture and Engineering Occupations",
"19": "Life, Physical, and Social Science Occupations",
"21": "Community and Social Service Occupations",
"23": "Legal Occupations",
"25": "Educational Instruction and Library Occupations",
"27": "Arts, Design, Entertainment, Sports, and Media Occupations",
"29": "Healthcare Practitioners and Technical Occupations",
"31": "Healthcare Support Occupations",
"33": "Protective Service Occupations",
"35": "Food Preparation and Serving Related Occupations",
"37": "Building and Grounds Cleaning and Maintenance Occupations",
"39": "Personal Care and Service Occupations",
"41": "Sales and Related Occupations",
"43": "Office and Administrative Support Occupations",
"45": "Farming, Fishing, and Forestry Occupations",
"47": "Construction and Extraction Occupations",
"49": "Installation, Maintenance, and Repair Occupations",
"51": "Production Occupations",
"53": "Transportation and Material Moving Occupations",
"55": "Military Specific Occupations"}

In [23]:
h1b_dviz_v2c['SOC_MAJOR_GROUP'] = h1b_dviz_v2c['SOC_MAJOR_GROUP'].map(soc_map_dict)
h1b_dviz_v2c.head(3)

,CASE_STATUS,CASE_SUBMITTED,DECISION_DATE,VISA_CLASS,EMPLOYMENT_START_DATE,EMPLOYMENT_END_DATE,EMPLOYER_NAME,EMPLOYER_STATE_MAPPED,AGENT_REPRESENTING_EMPLOYER,SOC_CODE,NAICS_CODE,FULL_TIME_POSITION,PW_UNIT_OF_PAY,PW_WAGE_LEVEL,WAGE_UNIT_OF_PAY,H1B_DEPENDENT,WILLFUL_VIOLATOR,WORKSITE_STATE,NORMALIZED_PW,NORMALIZED_WAGE_FROM,NORMALIZED_WAGE_TO,SOC_MAJOR_GROUP
0,CERTIFIED,2018-01-29,2018-02-02,H-1B,2018-07-28,2021-07-27,MICROSOFT CORPORATION,WA,N,15-1132,51121,Y,Year,Level II,Year,N,N,WA,112549.00000,143915.00000,143915.00000,Computer and Mathematical Occupations
1,CERTIFIED,2017-10-23,2017-10-27,H-1B,2017-11-06,2020-11-06,ERNST & YOUNG U.S. LLP,NJ,Y,13-2011,541211,Y,Year,Level II,Year,N,N,CA,79976.00000,100000.00000,100000.00000,Business and Financial Operations Occupations
2,CERTIFIED,2018-08-30,2018-09-06,H-1B,2018-09-10,2021-09-09,LOGIXHUB LLC,TX,N,15-1141,541511,Y,Year,Level II,Year,N,N,TX,77792.00000,78240.00000,78240.00000,Computer and Mathematical Occupations


<a href='#Table of Contents'>Back to Table of Contents</a>

<a id='NAICS_CODE'></a>
____
### 2b. Feature Selection / Creation - Part 2

##### Categorizing `NAICS_CODE`

Analagous to the `SOC_CODE`, `NAICS_CODE` classifies companies into different industries based on the similartiy of production processes. According to the official government manual on [`NAICS-CODE`](https://www.census.gov/eos/www/naics/2017NAICS/2017_NAICS_Manual.pdf), NAICS_CODE is divided into 20 broad Sections, 88 Divisions, 238 Groups and 420 Classes.

Also analaglous to the `SOC_CODE`, the first two digits of the code represent the
NAICS sectors that represent the general categories of economic activities. Therefore, I will classify all companies into the 20 different sectors. The mapping is as follows: 

- 11: Agriculture, Forestry, Fishing and Hunting
- 21: Mining, Quarrying, and Oil and Gas Extraction
- 22: Utilities
- 23: Construction
- 31-33: Manufacturing
- 42: Wholesale Trade
- 44-45: Retail Trade
- 48-49 Transportation and Warehousing
- 51: Information
- 52: Finance and Insurance
- 53: Real Estate and Rental and Leasing
- 54: Professional, Scientific, and Technical Services
- 55: Management of Companies and Enterprises
- 56: Administrative and Support and Waste Management and Remediation Services
- 61: Educational Services
- 62: Health Care and Social Assistance
- 71: Arts, Entertainment, and Recreation
- 72: Accommodation and Food Services
- 81: Other Services (except Public Administration)
- 92: Public Administration


**A. Initial Investigation**

- Initial Investigation reveals that there are some invalud NAICS_CODEs that are in the dataset. We will drop those rows from the dataset. 

In [33]:
# # Copy of original h1b_dviz_v2 into h1b_dviz_v3 to save the cleaned column
h1b_dviz_v3 = h1b_dviz_v2c.copy()

h1b_dviz_v3['NAICS_BROAD'] = h1b_dviz_v3.NAICS_CODE.str[:2]  # Broad Sectors
invalid_naics = np.where(h1b_dviz_v3['NAICS_BROAD'].isin(['10','12','19','15','13','99']))[0]

h1b_dviz_v3 = h1b_dviz_v3.drop(invalid_naics)
h1b_dviz_v3b = h1b_dviz_v3.reset_index(drop=True)

# # Verification 
h1b_dviz_v3b.shape

(615457, 23)

**B. Mapping NAICS_CODE back into industries for better intepretation**


In [35]:
naics_code_dict = {"11": "Agriculture, Forestry, Fishing and Hunting",
"21": "Mining, Quarrying, and Oil and Gas Extraction",
"22": "Utilities",
"23": "Construction",
"31": "Manufacturing",
"32": "Manufacturing",
"33": "Manufacturing",
"42": "Wholesale Trade",
"44": "Retail Trade",
"45": "Retail Trade",
"48": "Transportation and Warehousing",
"49": "Transportation and Warehousing",
"51": "Information",
"52": "Finance and Insurance",
"53": "Real Estate and Rental and Leasing",
"54": "Professional, Scientific, and Technical Services",
"55": "Management of Companies and Enterprises",
"56": "Administrative and Support and Waste Management and Remediation Services",
"61": "Educational Services",
"62": "Health Care and Social Assistance",
"71": "Arts, Entertainment, and Recreation",
"72": "Accommodation and Food Services",
"81": "Other Services (except Public Administration)",
"92": "Public Administration"}

In [36]:
h1b_dviz_v3b['NAICS_BROAD'] = h1b_dviz_v3b['NAICS_BROAD'].map(naics_code_dict)
h1b_dviz_v3b.head(2)

,CASE_STATUS,CASE_SUBMITTED,DECISION_DATE,VISA_CLASS,EMPLOYMENT_START_DATE,EMPLOYMENT_END_DATE,EMPLOYER_NAME,EMPLOYER_STATE_MAPPED,AGENT_REPRESENTING_EMPLOYER,SOC_CODE,NAICS_CODE,FULL_TIME_POSITION,PW_UNIT_OF_PAY,PW_WAGE_LEVEL,WAGE_UNIT_OF_PAY,H1B_DEPENDENT,WILLFUL_VIOLATOR,WORKSITE_STATE,NORMALIZED_PW,NORMALIZED_WAGE_FROM,NORMALIZED_WAGE_TO,SOC_MAJOR_GROUP,NAICS_BROAD
0,CERTIFIED,2018-01-29,2018-02-02,H-1B,2018-07-28,2021-07-27,MICROSOFT CORPORATION,WA,N,15-1132,51121,Y,Year,Level II,Year,N,N,WA,112549.00000,143915.00000,143915.00000,Computer and Mathematical Occupations,Information
1,CERTIFIED,2017-10-23,2017-10-27,H-1B,2017-11-06,2020-11-06,ERNST & YOUNG U.S. LLP,NJ,Y,13-2011,541211,Y,Year,Level II,Year,N,N,CA,79976.00000,100000.00000,100000.00000,Business and Financial Operations Occupations,"Professional, Scientific, and Technical Services"


**C. Delete original NAICS_CODE**

In [37]:
# # Drop original NAICS_CODE
h1b_dviz_v3b = h1b_dviz_v3b.drop('NAICS_CODE', axis=1)
h1b_dviz_v3b.shape

(615457, 22)

In [47]:
h1b_dviz_v3b.iloc[27:30]

,CASE_STATUS,CASE_SUBMITTED,DECISION_DATE,VISA_CLASS,EMPLOYMENT_START_DATE,EMPLOYMENT_END_DATE,EMPLOYER_NAME,EMPLOYER_STATE_MAPPED,AGENT_REPRESENTING_EMPLOYER,SOC_CODE,FULL_TIME_POSITION,PW_UNIT_OF_PAY,PW_WAGE_LEVEL,WAGE_UNIT_OF_PAY,H1B_DEPENDENT,WILLFUL_VIOLATOR,WORKSITE_STATE,NORMALIZED_PW,NORMALIZED_WAGE_FROM,NORMALIZED_WAGE_TO,SOC_MAJOR_GROUP,NAICS_BROAD
27,CERTIFIED,2018-03-20,2018-03-26,H-1B,2018-09-19,2021-09-18,TWO95 INTERNATIONAL,NJ,Y,15-1132,Y,Year,Level I,Year,Y,N,PA,70221.00000,72000.00000,72000.00000,Computer and Mathematical Occupations,"Professional, Scientific, and Technical Services"
28,CERTIFIED-WITHDRAWN,2018-03-26,2018-05-03,H-1B,2018-07-29,2021-07-28,"RESONETICS, LLC",NH,Y,17-2071,Y,Year,Level III,Year,N,N,NH,104998.00000,105000.00000,115000.00000,Architecture and Engineering Occupations,"Professional, Scientific, and Technical Services"
29,CERTIFIED,2018-03-12,2018-03-16,H-1B,2018-09-11,2021-09-11,ALTIMETRIK CORP,MI,Y,15-1132,Y,Year,Level II,Year,Y,N,MI,78874.00000,78874.00000,95000.00000,Computer and Mathematical Occupations,"Professional, Scientific, and Technical Services"


<a href='#Table of Contents'>Back to Table of Contents</a>

<a id='Cleaned Dataset Output'></a>
____

### 3. Saving the Cleaned Dataset

To learn more, refer to previous notebook (ch.5) 


**A. Function from previous notebook to get dataframe column types**

In [38]:
def get_col_dtypes(df):
    dtypes = df.dtypes

    col_name = dtypes.index                      # name of the column 
    col_types = [i.name for i in dtypes.values]  # datatype of the column

    # # Save column name and types into a dictionary 
    # # {name: type}
    column_types = dict(zip(col_name, col_types))
    return column_types

**B. Save final dataframe to a csv file**

In [39]:
# # Cleaned Dataset (with optimized datatypes)
h1b_dviz_v3b.to_csv("data/H-1B_Feature_Creation_Clean.csv", index=False)

**C. Save necessary parameter info into a dictionary (column types / final columns etc.)**

In [40]:
# # LIST of final columns
final_columns = list(h1b_dviz_v3b.columns)

In [41]:
# # LIST of date columns
# # Also a TUPLE of date columns so that I can remove these keys from dictionary
date_col_list = ['CASE_SUBMITTED', 'DECISION_DATE', 'EMPLOYMENT_START_DATE', 'EMPLOYMENT_END_DATE']
date_col_tuple = tuple(date_col_list)   # Remember to use 'date_col_list' 
                                        # date_columns is already defined above from the cleaning step

In [42]:
# # DICTIONARY of {column: column types} for h1b cleaned dataset (using function above)
col_type_dict = get_col_dtypes(h1b_dviz_v3b)

In [43]:
# # Column dtypes dictionary without date columns
for k in date_col_tuple:
    col_type_dict.pop(k, None)
    
# len(col_type_dict)
# col_type_dict

In [44]:
# # Create dictionary for JSON file format conversion

cleaned_params_dict = {}

cleaned_params_dict['col_types'] = col_type_dict
cleaned_params_dict['date_cols'] = date_col_list
cleaned_params_dict['final_cols'] = final_columns

**D. Save parameters into a JSON file**


In [45]:
# convert to json format
import json 

with open('data/H-1B_Feature_Creation_Cleaned_Params.json', 'w') as fp:
    json.dump(cleaned_params_dict, fp)  # convert dict as a json file format! 

#### END OF NOTEBOOK
<a href='#Top'>Back to Top</a>